In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import random
import json
import zipfile
import io
import time
from collections import deque

# ============================================================================
# 1. TIC-TAC-TOE ENVIRONMENT (Exact Copy)
# ============================================================================
class TicTacToe:
    def __init__(self, grid_size=3, win_length=None):
        self.grid_size = grid_size
        self.win_length = win_length if win_length else grid_size
        self.reset()
    
    def reset(self):
        self.board = np.zeros((self.grid_size, self.grid_size), dtype=int)
        self.current_player = 1
        self.game_over = False
        self.winner = None
        self.move_history = []
        return self.get_state()
    
    def get_state(self):
        return tuple(self.board.flatten())
    
    def get_available_actions(self):
        actions = [(r, c) for r in range(self.grid_size) 
                   for c in range(self.grid_size) if self.board[r, c] == 0]
        
        # --- TOURNAMENT RULE FIX (Crucial for 4x4) ---
        if len(self.move_history) == 0:
            center = self.grid_size // 2
            if self.grid_size % 2 == 1:
                if (center, center) in actions: actions.remove((center, center))
            else:
                forbidden = [(center-1, center-1), (center-1, center), 
                             (center, center-1), (center, center)]
                actions = [a for a in actions if a not in forbidden]
        return actions

    def make_move(self, position):
        if self.game_over: return self.get_state(), 0, True
        i, j = position
        if self.board[i, j] != 0: return self.get_state(), -10, True
        
        self.board[i, j] = self.current_player
        self.move_history.append((position, self.current_player))
        
        if self._check_win(self.current_player):
            self.game_over = True; self.winner = self.current_player
            return self.get_state(), 100, True
        
        if len(self.get_available_actions()) == 0:
            self.game_over = True; self.winner = 0 
            # Defender Bonus
            if self.current_player == 2: return self.get_state(), 50, True 
            else: return self.get_state(), 0, True 

        self.current_player = 3 - self.current_player
        return self.get_state(), 0, False
    
    def _check_win(self, player):
        board = self.board; n = self.grid_size; w = self.win_length
        for r in range(n):
            for c in range(n - w + 1):
                if np.all(board[r, c:c+w] == player): return True
        for r in range(n - w + 1):
            for c in range(n):
                if np.all(board[r:r+w, c] == player): return True
        for r in range(n - w + 1):
            for c in range(n - w + 1):
                if np.all([board[r+k, c+k] == player for k in range(w)]): return True
                if np.all([board[r+k, c+w-1-k] == player for k in range(w)]): return True
        return False
    
    def evaluate_position(self, player):
        if self.winner == player: return 100000
        if self.winner == (3 - player): return -100000
        if self.game_over: return 0 
        
        opponent = 3 - player
        score = 0
        is_large_grid = self.grid_size > 3
        is_defensive_agent = (player == 2)
        
        # IRON WALL Strategy weights
        if is_large_grid and is_defensive_agent:
            threat_2_weight = 400; threat_win_minus_1 = 8000
            my_attack_weight = 10; center_control_penalty = 200
        else:
            threat_2_weight = 60; threat_win_minus_1 = 1000
            my_attack_weight = 50; center_control_penalty = 50

        # Center Control
        center = self.grid_size // 2
        centers = []
        if self.grid_size % 2 == 0:
            centers = [(center-1, center-1), (center-1, center), (center, center-1), (center, center)]
        else: centers = [(center, center)]

        for r, c in centers:
            if self.board[r, c] == player: score += 50
            elif self.board[r, c] == opponent: 
                score -= 50
                if is_defensive_agent: score -= center_control_penalty

        # Corner Control
        corners = [(0,0), (0, self.grid_size-1), (self.grid_size-1, 0), (self.grid_size-1, self.grid_size-1)]
        for r,c in corners:
            if self.board[r,c] == player: score += 10
            elif self.board[r,c] == opponent: score -= 10

        # Simplified heuristics for training speed
        return score

# ============================================================================
# 2. STRATEGIC AGENT
# ============================================================================
class StrategicAgent:
    def __init__(self, player_id, lr=0.2, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01):
        self.player_id = player_id; self.lr = lr; self.gamma = gamma
        self.epsilon = epsilon; self.epsilon_decay = epsilon_decay; self.epsilon_min = epsilon_min
        self.q_table = {}
        self.minimax_depth = 4
        self.wins = 0; self.losses = 0; self.draws = 0
    
    def get_q_value(self, state, action):
        return self.q_table.get((state, action), 0.0)
    
    def choose_action(self, env, training=True):
        available_actions = env.get_available_actions()
        if not available_actions: return None
        
        # LEVEL 1: Survival
        for action in available_actions:
            sim = self._simulate_move(env, action, self.player_id)
            if sim.winner == self.player_id: return action
        opponent = 3 - self.player_id
        for action in available_actions:
            sim = self._simulate_move(env, action, opponent)
            if sim.winner == opponent: return action 

        # LEVEL 2: Strategic Planning
        if training and random.random() < self.epsilon:
            return random.choice(available_actions)

        # Dynamic Depth Logic
        empty_spots = len(available_actions)
        depth_bonus = 2 if (env.grid_size > 1 and self.player_id == 2) else 0
        current_depth = min(self.minimax_depth + depth_bonus, empty_spots)
        if env.grid_size == 3: current_depth = 9

        best_score = -float('inf'); best_actions = []
        center = env.grid_size // 2
        available_actions.sort(key=lambda x: abs(x[0]-center) + abs(x[1]-center))
        alpha = -float('inf'); beta = float('inf')

        for action in available_actions:
            sim_env = self._simulate_move(env, action, self.player_id)
            score = self._minimax(sim_env, current_depth - 1, alpha, beta, False)
            # Q-Table Tiebreaker
            q_boost = self.get_q_value(env.get_state(), action) * 0.01
            total_score = score + q_boost

            if total_score > best_score:
                best_score = total_score; best_actions = [action]
            elif total_score == best_score:
                best_actions.append(action)
            alpha = max(alpha, best_score)
        
        return random.choice(best_actions) if best_actions else random.choice(available_actions)

    def _minimax(self, env, depth, alpha, beta, is_maximizing):
        if env.winner == self.player_id: return 1000 + depth
        if env.winner == (3 - self.player_id): return -1000 - depth
        if env.game_over: return 0 
        if depth == 0: return env.evaluate_position(self.player_id)
        available_actions = env.get_available_actions()
        if is_maximizing:
            max_eval = -float('inf')
            for action in available_actions:
                sim_env = self._simulate_move(env, action, self.player_id)
                eval = self._minimax(sim_env, depth - 1, alpha, beta, False)
                max_eval = max(max_eval, eval); alpha = max(alpha, eval)
                if beta <= alpha: break
            return max_eval
        else:
            min_eval = float('inf'); opponent = 3 - self.player_id
            for action in available_actions:
                sim_env = self._simulate_move(env, action, opponent)
                eval = self._minimax(sim_env, depth - 1, alpha, beta, True)
                min_eval = min(min_eval, eval); beta = min(beta, eval)
                if beta <= alpha: break
            return min_eval

    def _simulate_move(self, env, action, player):
        sim = TicTacToe(env.grid_size, env.win_length)
        sim.board = env.board.copy(); sim.current_player = player
        sim.make_move(action)
        return sim

    def update_q_value(self, state, action, reward, next_state, next_available_actions):
        current_q = self.get_q_value(state, action)
        max_next = max([self.get_q_value(next_state, a) for a in next_available_actions]) if next_available_actions else 0
        self.q_table[(state, action)] = current_q + self.lr * (reward + self.gamma * max_next - current_q)

    def decay_epsilon(self):
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
    
    def reset_stats(self):
        self.wins = 0; self.losses = 0; self.draws = 0

def play_game(env, agent1, agent2, training=True):
    env.reset(); history = []
    agents = {1: agent1, 2: agent2}
    while not env.game_over:
        cp = env.current_player; ag = agents[cp]
        s = env.get_state(); a = ag.choose_action(env, training)
        if a is None: break
        history.append((s, a, cp))
        ns, r, d = env.make_move(a)
        if training: ag.update_q_value(s, a, r, ns, env.get_available_actions())
        if d:
            if env.winner == 1:
                agent1.wins+=1; agent2.losses+=1
                if training: _update_outcome(agent1, history, 1, 100); _update_outcome(agent2, history, 2, -50)
            elif env.winner == 2:
                agent2.wins+=1; agent1.losses+=1
                if training: _update_outcome(agent1, history, 1, -50); _update_outcome(agent2, history, 2, 100)
            else:
                agent1.draws+=1; agent2.draws+=1
                if training: _update_outcome(agent1, history, 1, -5); _update_outcome(agent2, history, 2, -5)
    return env.winner

def _update_outcome(agent, history, pid, reward):
    moves = [(s, a) for s, a, p in history if p == pid]
    for i in range(len(moves)-1, -1, -1):
        s, a = moves[i]
        disc = agent.gamma ** (len(moves)-1-i)
        curr = agent.get_q_value(s, a)
        agent.q_table[(s, a)] = curr + agent.lr * (reward * disc - curr)

In [3]:
# ============================================================================
# 1. SERIALIZATION (Matches Streamlit Inputs EXACTLY)
# ============================================================================
def serialize_q_table(q_table):
    """Safe serialization for JSON keys"""
    serialized_q = {}
    for (state, action), value in q_table.items():
        state_list = [int(x) for x in state]
        action_list = [int(x) for x in action]
        # Exact key format: JSON string of Tuple(List, List)
        key_str = json.dumps((state_list, action_list))
        serialized_q[key_str] = float(value)
    return serialized_q

def save_kaggle_brain(agent1, agent2, config, filename="agi_agents.zip"):
    # Prepare Data Dictionaries
    agent1_state = {
        "q_table": serialize_q_table(agent1.q_table),
        "epsilon": agent1.epsilon, "lr": agent1.lr, 
        "gamma": agent1.gamma, "wins": agent1.wins, 
        "losses": agent1.losses, "draws": agent1.draws
    }
    agent2_state = {
        "q_table": serialize_q_table(agent2.q_table),
        "epsilon": agent2.epsilon, "lr": agent2.lr, 
        "gamma": agent2.gamma, "wins": agent2.wins, 
        "losses": agent2.losses, "draws": agent2.draws
    }
    
    print(f"💾 Saving: A1 Q-States={len(agent1.q_table)}, A2 Q-States={len(agent2.q_table)}")
    with zipfile.ZipFile(filename, "w", zipfile.ZIP_DEFLATED) as zf:
        zf.writestr("agent1.json", json.dumps(agent1_state))
        zf.writestr("agent2.json", json.dumps(agent2_state))
        zf.writestr("config.json", json.dumps(config))
    print(f"✅ Save Complete! Download '{filename}'")

# ============================================================================
# 2. TRAINING RUNNER: SERIOUS MODE
# ============================================================================
def run_serious_training():
    # --- CONFIGURATION FOR SERIOUS 4x4 PLAY ---
    EPISODES = 200      # 10k episodes for 4x4 complexity
    GRID_SIZE = 4
    WIN_LENGTH = 3        # As you requested
    
    print(f"🚀 Starting SERIOUS Mode Training ({GRID_SIZE}x{GRID_SIZE}, Win {WIN_LENGTH})")
    print(f"🎯 Target: {EPISODES} Episodes...")
    
    # Initialize Environment
    env = TicTacToe(GRID_SIZE, WIN_LENGTH)
    
    # Initialize Agents (Optimized for Larger Grid)
    agent1 = StrategicAgent(1, lr=0.2, gamma=0.95, epsilon_decay=0.9995)
    agent2 = StrategicAgent(2, lr=0.2, gamma=0.95, epsilon_decay=0.9995)
    
    # Increase Thinking Depth for Training
    agent1.minimax_depth = 5
    agent2.minimax_depth = 5
    
    # --- FIX: Initialize History Lists ---
    history = {
        'agent1_wins': [], 'agent2_wins': [], 'draws': [],
        'agent1_epsilon': [], 'agent2_epsilon': [],
        'agent1_q_size': [], 'agent2_q_size': [],
        'episode': []
    }
    
    start_time = time.time()
    
    # Training Loop
    for ep in range(1, EPISODES + 1):
        play_game(env, agent1, agent2, training=True)
        
        agent1.decay_epsilon()
        agent2.decay_epsilon()
        
        # Record stats every 50 episodes (prevents huge file size)
        if ep % 50 == 0:
            history['agent1_wins'].append(agent1.wins)
            history['agent2_wins'].append(agent2.wins)
            history['draws'].append(agent1.draws)
            history['agent1_epsilon'].append(agent1.epsilon)
            history['agent2_epsilon'].append(agent2.epsilon)
            history['agent1_q_size'].append(len(agent1.q_table))
            history['agent2_q_size'].append(len(agent2.q_table))
            history['episode'].append(ep)
        
        if ep % 10 == 0:
            elapsed = time.time() - start_time
            print(f"Ep {ep}/{EPISODES} | P1 Wins: {agent1.wins} | P2 Wins: {agent2.wins} | ε={agent1.epsilon:.4f} | {elapsed:.1f}s")
            
    # Save Final Brain with FULL history
    config = {
        "grid_size": GRID_SIZE,
        "win_length": WIN_LENGTH,
        "lr1": agent1.lr, "gamma1": agent1.gamma,
        "lr2": agent2.lr, "gamma2": agent2.gamma,
        "minimax_depth1": agent1.minimax_depth,
        "minimax_depth2": agent2.minimax_depth,
        "training_history": history  # The fix is here!
    }
    
    print("\n🏆 Training Finished!")
    save_kaggle_brain(agent1, agent2, config)

if __name__ == "__main__":
    run_serious_training()

🚀 Starting SERIOUS Mode Training (4x4, Win 3)
🎯 Target: 200 Episodes...
Ep 10/200 | P1 Wins: 8 | P2 Wins: 2 | ε=0.9950 | 0.3s
Ep 20/200 | P1 Wins: 13 | P2 Wins: 7 | ε=0.9900 | 1.6s
Ep 30/200 | P1 Wins: 17 | P2 Wins: 13 | ε=0.9851 | 1.9s
Ep 40/200 | P1 Wins: 22 | P2 Wins: 18 | ε=0.9802 | 12.4s
Ep 50/200 | P1 Wins: 31 | P2 Wins: 19 | ε=0.9753 | 12.7s
Ep 60/200 | P1 Wins: 35 | P2 Wins: 25 | ε=0.9704 | 22.3s
Ep 70/200 | P1 Wins: 42 | P2 Wins: 28 | ε=0.9656 | 22.6s
Ep 80/200 | P1 Wins: 49 | P2 Wins: 31 | ε=0.9608 | 22.9s
Ep 90/200 | P1 Wins: 52 | P2 Wins: 38 | ε=0.9560 | 23.2s
Ep 100/200 | P1 Wins: 56 | P2 Wins: 44 | ε=0.9512 | 36.2s
Ep 110/200 | P1 Wins: 63 | P2 Wins: 47 | ε=0.9465 | 36.5s
Ep 120/200 | P1 Wins: 71 | P2 Wins: 49 | ε=0.9418 | 71.9s
Ep 130/200 | P1 Wins: 76 | P2 Wins: 54 | ε=0.9371 | 161.6s
Ep 140/200 | P1 Wins: 82 | P2 Wins: 58 | ε=0.9324 | 165.6s
Ep 150/200 | P1 Wins: 89 | P2 Wins: 61 | ε=0.9277 | 245.6s
Ep 160/200 | P1 Wins: 96 | P2 Wins: 64 | ε=0.9231 | 310.4s
Ep 170/200 